# Introduction to Computer Programming and Numerical Methods

> **Mohamad M. Hallal, PhD** <br> Teaching Professor, UC Berkeley

[![License](https://img.shields.io/badge/license-CC%20BY--NC--ND%204.0-blue)](https://creativecommons.org/licenses/by-nc-nd/4.0/)
***

# Root Finding

1. [**Root Finding**](#s1)
2. [**Bisection Method**](#s2)
3. [**Newton-Raphson Method**](#s3)
4. [**Root Finding in Python**](#s4)

***

# 0. Motivation

Much like $\sin(x)$, $\cos(x)$, $e^x$, and $\log(x)$, which cannot be explicitly computed, analytically solving many equations in engineering and science is not possible. The goal of equation solving is to find the value of the independent variable, generally referred to as $x$, that makes the equation true. For simple functions such as $f(x) = ax^2 + bx + c = 0$, you may already be familiar with the quadratic formula, which provides the exact solutions:

$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

However for more complicated functions, the solution can rarely be computed using such explicit, or exact, means. Several numerical methods exist to find approximate solutions to many functions commonly encountered in engineering and science. Numerical methods mainly focus on approximating the solutions to equations without relying on specific formulas or other algebraic techniques. The advantages of these numerical techniques include their applicability to nearly any function, their quick and simple implementation, and the minimal mathematical knowledge that is required. However, as with other numerical methods, the solutions obtained will be approximations and are subject to errors.

**Learning objectives:**

* Rewrite a nonlinear equation as a root finding problem
* Describe the general approach of numerical root finding algorithms, including concepts such as error, tolerance, and convergence
* Discuss why tolerance is used as opposed to zero error
* Explain the bisection method, its applicability, and limitations
* Implement the bisection method in Python
* Explain the Newton-Raphson method, its applicability, and limitations
* Implement the Newton-Raphson method in Python
* Use existing root finding functions in Python, particularly `fsolve()` from `scipy.optimize`

# 1. Root Finding <a id="s1"></a>

Let's first take a look at the general idea of solving a nonlinear equation. Consider the equation $\ell(x)=r(x)$, where $\ell(x)$ is the left-hand expression and $r(x)$ is the right-hand expression. To find the values of $x$ for which this equation is true, which we will refer to as equation solving, we can first rewrite it by subtracting the right-hand side from the left to get:

$$\ell(x)-r(x)=0$$

If we define $f(x) = \ell(x)-r(x)$, then solving the equation $\ell(x)=r(x)$ is essentially the same as solving $f(x)=0$. This is true for any equation. Thus, in the remainder of this section, we focus on solving equations of the form $f(x)=0$.

A **root** of a function $f(x)$ is a value $x_r$ that satisfies $f(x_r) = 0$. A function may possess zero, one, or multiple roots. For instance, in the case of the function $f(x) = x^2 - 9$, it is evident that the roots are precisely $3$ and $-3$. However, consider a more complex function like $f(x)=e^{-40x}+40xe^{-40x}-0.05$. It should be evident that finding the roots of this equation is not possible using specific formulas or other algebraic techniques. As such, when dealing with more complex functions, determining an analytical or exact solution for its roots can prove to be challenging. In such situations, the application of numerical methods to find approximate solutions becomes invaluable.

Numerical methods for root finding are typically iterative in nature and can also be implemented recursively. These methods involve a series of steps, including:

1. Initiate an initial guess, denoted as $x_0$
2. Iterate to refine the answer $x_i$
3. Terminate when $x_i$ reaches an acceptable approximation

For some equations, the roots can extend to infinite decimal places. Consequently, when numerically computing them, it is crucial to establish a metric for quantifying the error and a criterion for what defines an acceptable approximation. 

## 1.1. Error

Error quantifies the deviation between an expected value and the computed value. In root finding, if $x_i$ is the estimated root at iteration $i$ and $x_r$ is the actual exact root, where $f(x_r) = 0$, error can be measured in various ways:

* Absolute value of the function at $x_i$: $|f(x_i)|$
<br>

* Absolute error: $|x_r - x_i|$
<br>

* Relative error: $\dfrac{|x_r - x_i|}{|x_r|}$

However, in practice, the exact root $x_r$ is generally unknown, as it is the value we are attempting to determine. Consequently, calculating absolute and relative error using $x_r$ is impractical. Root finding methods operate iteratively and gradually approach the root. As the estimate $x_i$ approaches $x_r$, the change between successive iterations, $|x_{i+1} - x_i|$, diminishes. As such, the absolute and relative error can be estimated as:
* Absolute error: $|x_{i+1} - x_i|$
<br>

* Relative error: $\dfrac{|x_{i+1} - x_i|}{|x_{i+1}|}$

<div class="alert alert-block alert-warning"> <b>NOTE!</b> These different measures of error have their advantages and disadvantages, and the choice of which metric to use must be done very carefully.</div>

All of the above error definitions will be exactly equal to 0 if $x_i=x_r$. However, achieving  exactly zero error is often impractical and, in many cases, impossible due to the finite precision of computer arithmetic. Computers use floating-point numbers, resulting in small numerical fluctuations even when extremely close to the actual root. Therefore, exact zero error is rarely attainable in practical numerical computations. Instead of aiming for zero error, the goal is to find an acceptable approximation $x_i$ such that $|f(x_i)| \approx 0$, or equivalently, $|x_r - x_i| \approx 0$.

## 1.2. Tolerance

Numerical algorithms use tolerance thresholds to decide when to stop iterations. Tolerance defines the acceptable level of error for an application. Starting with an initial guess $x_0$, iterations are used to refine the estimate until the error is below a predefined tolerance, $\epsilon$. At this point, the iterations are terminated and the value $x_i$ is deemed an acceptable approximation of the root. This stopping criterion ensures practical accuracy while accounting for computational limitations. Another stopping criterion is when the number of iterations reaches a predetermined maximum number. As such, the common stopping criteria are:
1. $|f(x_i)| \leq \epsilon $ 
<br>

2. $|x_{i+1} - x_i| \leq \epsilon$
<br>
3. $\dfrac{|x_{i+1} - x_i|}{|x_{i+1}|} \leq \epsilon$
<br>
4. $i = N$, where $N$ is a predetermined maximum number of iterations

Selecting a suitable tolerance is critical, as it directly impacts both the accuracy and computational cost of an algorithm.

## 1.3. Convergence

Convergence occurs when a numerical method iteratively refines an initial guess $x_0$ until the error falls below the predefined tolerance. Specifically, a solution is said to have converged when the iteration yields $x_i$ with an error smaller than the tolerance.

We now have a way to view every equation-solving problem as well as a way to define stopping criteria. Next, we will introduce different numerical techniques for root finding.

# 2. Bisection Method <a id="s2"></a>

One common numerical method for root finding is the bisection method. As its name suggests, the method is based on repeatedly bisecting (dividing into two equal parts) an interval that contains a root. The basic idea is very simple and is grounded in the Intermediate Value Theorem.

The Intermediate Value Theorem states that if two points are connected by a continuous curve, with one point below a certain line and the other point above a certain line, then there is at least one place where the curve crosses the line.

<br>

<center><figure>
  <img src="https://www.mathsisfun.com/algebra/images/intermediate-value-a.svg
" style="width:25%">
    <figcaption style="text-align:center"><strong><br>Intermediate Value Theorem:</strong> <a href="https://www.mathsisfun.com/algebra/intermediate-value-theorem.html">https://www.mathsisfun.com/</a></figcaption>   
</figure></center>

<br>

Now if this line corresponds to $y=0$, then the curve would cross the line at a root.  

Suppose $f(x)$ is a continuous function on a closed interval $[a, b]$. If $f(a)$ and $f(b)$ have opposite signs (one is positive and the other is negative), then there must exits at least one root within $[a, b]$. This is because if the function is continuous, then to go from negative to positive (or vice versa), the function must cross through $f(x)=0$.

The steps of the bisection method are as follows:

1. Choose an initial interval $[a, b]$ where the function $f(x)$ changes signs: $f(a) \times f(b) < 0$
<br>
2. Compute the midpoint of the interval: $m = \dfrac{a + b}{2}$
<br>
3. Evaluate the function at the midpoint: $f(m)$ 
<br>
4. Check the error

    4.1. If $|f(m)| \leq \epsilon$ (or any other equivalent error is within the tolerance), then $m$ is an acceptable root
    
    4.2. Else if $f(a) \times f(m) < 0$, then the root lies in the interval $[a, m]$. Set $b = m$.
    
    4.3. Else, the root lies in the interval $[m, b]$. Set $a = m$.
<br>
4. Repeat the process with the new interval until the stopping criterion is satisfied

<br>

<center><figure>
  <img src="https://x-engineer.org/wp-content/uploads/2016/11/The-Bisection-Method-animation.gif
" style="width:50%">
    <figcaption style="text-align:center"><strong>Animation of the bisection method:</strong> <a href="https://x-engineer.org/bisection-method/">https://x-engineer.org/</a></figcaption>   
</figure></center>

<br>

<div class="alert alert-block alert-info"> <b>TRY IT!</b> Write a recursive function <code>myBisection(f, a, b, tol)</code> which takes as input a function <code>f</code> and three scalar values <code>a, b, tol</code> and returns a root <code>m</code> of <code>f</code>, bounded by $a$ and $b$ to within $|f(m)| \leq {\text{tol}}$. If $f(a)\times f(b) <0$ is not satisfied, raise an exception with the message <code>'Invalid [a, b]'</code> using an <code>assert</code> statement.</div>

In [ ]:


# Define function

    # check if a and b are valid
    
        
    # get midpoint
    
    # check error
    

<div class="alert alert-block alert-info"> <b>TRY IT!</b> Try your function <code>myBisection(f, a, b, tol)</code> for $f(x)=e^{-40x}+40xe^{-40x}-0.05$ with different values of <code>a, b, tol</code>.</div>

In [ ]:
import numpy as np

f = lambda x: np.exp(-40*x) + 40*x*np.exp(-40*x) - 0.05
root = myBisection(f, 0.1, 0.2, 1e-15)
print(f'r    = {root}')
print(f'f(r) = {f(root)}')

The bisection method works because it systematically narrows down the interval containing the root. By continually halving the interval and choosing the subinterval where the sign change occurs, it ensures that the root is always within the interval. This method is guaranteed to converge to a root, provided that the initial interval $[a,b]$ is chosen such that $f(a)$ and $f(b)$ have opposite signs.

Despite its simple algorithm and guaranteed convergence under certain conditions, the method can be slow compared to other root finding methods, especially if the interval is large. In addition, the method fails to find a solution if the signs of the function on either side of the root are the same.

# 3. Newton-Raphson Method <a id="s3"></a>

The quality of numerical root finding algorithms can be greatly improved by leveraging some calculus. One such method is the Newton-Raphson method, also known as Newton's method. The method was originally proposed by Isaac Newton, and later modified by Joseph Raphson. The method uses the idea that a continuous and differentiable function can be approximated by a straight line tangent to it. 

Consider the plot below. As we approach the root, the function can be approximated by a straight line (i.e., a linear function). So, if we take any point along the function in the zoomed-in view, and draw a tangent line, the $x$-intercept of the tangent line (where it intersects the $x$ axis) is a reasonable estimate of the root. By repeating this process multiple times, the estimate can be refined such that it becomes an acceptable approximation of the root.

<br>

<center><figure>
  <img src="https://docs.google.com/drawings/d/e/2PACX-1vSFnZxT8BSCI34RQfZhJXlwY-HE1tUUZUW7vzNOfWuxVfjC3BbCJIVRZpb8u2lCjhbJT2yx3smQqnf3/pub?w=1270&h=480
" style="width:75%">
    <figcaption style="text-align:center"><strong><br>Zoomed-in view of a root</strong></figcaption>   
</figure></center>

<br>

Using calculus, we can show that if we draw a tangent line at a point $x_{i-1}$, where the slope of that tangent line is simply the value of the first derivative of the function at that point, $f'(x_{i-1})$, then the tangent line intersects the $x$-axis at:

$$x_i = x_{i-1} - \dfrac{f(x_{i-1})}{f'(x_{i-1})}$$

<br>

<center><figure>
  <img src="https://docs.google.com/drawings/d/e/2PACX-1vRWq2sfHb5Y0aDtHAdC9S0MxvxPvtjLFwzi3P0LTuz2W0I-LDqellEra4EFUFYIyZMEWDzHDP2zbauP/pub?w=885&h=527
" style="width:50%">
    <figcaption style="text-align:center"><strong><br>Calculating $x$-intercept of the tangent line</strong></figcaption>   
</figure></center>

<br>

The steps of the Newton-Raphson method are as follows:

1. Choose an initial guess $x_0$
<br>
2. Evaluate the function at $x_{i}$: $f(x_i)$
<br>
3. Check the error

    3.1. If $|f(x_i)| \leq \epsilon$ (or any other equivalent error is within the tolerance), then $x_i$ is an acceptable root
    
    3.2. Else calculate an improved estimate $x_{i}$ using the formula: $x_{i} = x_{i-1} - \dfrac{f(x_{i-1})}{f'(x_{i-1})}$
   
   
4. Repeat the process with the new estimate until the stopping criterion is satisfied

<br>

<center><figure>
  <img src="https://yasincapar.com/wp-content/uploads/2020/07/newto2n.gif
" style="width:40%">
    <figcaption style="text-align:center"><strong><br>Animation of the Netwon-Raphson method:</strong> <a href="https://yasincapar.com/the-newton-raphson-method/">https://yasincapar.com/</a></figcaption>   
</figure></center>

<br>

<div class="alert alert-block alert-info"> <b>TRY IT!</b> Write a recursive function <code>myNewton(f, df, x0, tol)</code> which takes as input two functions <code>f</code> and its derivative <code>df</code> and two scalar values <code>x0, tol</code> and returns a root <code>r</code> of <code>f</code> such that $|f(r)| \leq {\text{tol}}$.</div>

In [ ]:
# Define function

    # check error
    

<div class="alert alert-block alert-info"> <b>TRY IT!</b> Try your function <code>myNewton(f, df, x0, tol)</code> for $f(x)=e^{-40x}+40xe^{-40x}-0.05$ with different values of <code>x0, tol</code>.</div>

In [ ]:
f = lambda x: np.exp(-40*x) + 40*x*np.exp(-40*x) - 0.05
df = lambda x: -40*np.exp(-40*x) + 40*np.exp(-40*x) - 40*40*x*np.exp(-40*x)
root = myNewton(f, df, 0.15, 1e-5)
print(f'r    = {root}')
print(f'f(r) = {f(root)}')

<div class="alert alert-block alert-info"> <b>TRY IT!</b> Try your function <code>myNewton(f, df, x0, tol)</code> for $f(x)=e^{-40x}+40xe^{-40x}-0.05$ with <code>x0 = 0.18</code>. Then run the next code to visualize why the method fails to find the desired root.</div>

In [ ]:
root = myNewton(f, df, 0.18, 1e-5)
print(f'r    = {root}')
print(f'f(r) = {f(root)}')

In [ ]:
import matplotlib.pyplot as plt

# Create figure and axes
fig, ax = plt.subplots(figsize=(5,3), dpi=300)

# control axes
for spine in ['left', 'bottom']:
    ax.spines[spine].set_position('zero')

for spine in ['right', 'top']:
    ax.spines[spine].set_color('none')

ax.axis([-0.2, 0.2, -0.2, 1])

plt.rcParams['xtick.major.pad']='5'
plt.rcParams['ytick.major.pad']='5'

ax.set(yticks = [])
ax.set(xticks = [-0.2, -0.1, 0.1, 0.2])

# define array of x values for plotting
x = np.linspace(-0.2, 0.2,1000)

# define function that returns the value of the tangent to a function evaluated at point x0
tangent = lambda x0, x: df(x0)*x + f(x0) - df(x0)*x0

# plot the function
plt.plot(x, f(x), 'C5', lw=2.5)

# plot the tangent
plt.plot(np.linspace(-0.2, 0.18, 1000), tangent(0.18, np.linspace(-0.2, 0.18, 1000)), lw=1)
plt.plot(0.18, f(0.18), 'oC0')

plt.show()

The Newton-Raphson method estimates the root by repeatedly refining an initial guess based on the function and its derivative. As opposed to the bisection method, which required two initial guesses $[a, b]$ with different function signs, the Newton-Raphson method only requires one initial guess. In addition, the method converges to a root much faster than the bisection method if the initial guess is close to the actual root.

However, it should be clear that this method is applicable only if the function is both continuous and differentiable, whereas the bisection method only requires the function to be continuous. In addition, the Newton-Raphson method requires dividing by the derivative of the function. Consequently, if the derivative at $x_i$ is zero or close to 0, this will result in division by zero, and the method will fail to converge to a solution.

# 4. Root Finding in Python <a id="s4"></a>

Several Python modules have existing root finding functions. One such function is `fsolve()` from `scipy.optimize`. To use it, we will need to import SciPy: `import scipy`. The `scipy.optimize.fsolve()` function takes in many arguments that you can read about in the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html). The most important two are the function you want to find its root, `f`, and the initial guess, `x0`: `scipy.optimize.fsolve(f, x0)`. 

<div class="alert alert-block alert-info"> <b>TRY IT!</b> Use <code>fsolve(f, x0)</code> to find a root for $f(x)=e^{-40x}+40xe^{-40x}-0.05$ with different values of <code>x0</code>.</div>

In [ ]:
f = lambda x: np.exp(-40*x) + 40*x*np.exp(-40*x) - 0.05

# find a root using fsolve()
